In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pyvis.network import Network
import networkx.algorithms.bipartite as bipartite

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('./global_terror_data_proj_1.csv')

In [12]:
def trim_edges(g, weight=1):
        g2=nx.Graph()
        for f, to, edata in g.edges(data=True):
                if edata['weight'] > weight:
                        g2.add_edge(f,to,edata)
        return g2

In [13]:
def island_method(g, iterations=5):
    weights= [edata['weight'] for f,to,edata in g.edges(data=True)]

    mn=int(min(weights))
    mx=int(max(weights))
    #compute the size of the step, so we get a reasonable step in iterations
    step=int((mx-mn)/iterations)

    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx,step)]

In [3]:
ter_df = df[["country_txt", "gname"]]
G=nx.from_pandas_dataframe(ter_df, "country_txt", "gname")

In [4]:
print (nx.info(G))

Name: 
Type: Graph
Number of nodes: 3046
Number of edges: 4249
Average degree:   2.7899


In [5]:
me_list = ["Syria", "Turkey", "Iraq","Jordan", 
           "Pakistan", "Afghanistan","Iran", "Lebanon"]
ter_df =  ter_df[ter_df.country_txt.isin(me_list)]
G=nx.from_pandas_dataframe(ter_df,
                               "country_txt", 
                               "gname")
ter_df.head()

,country_txt,gname
59,Lebanon,Relatives of terrorist
60,Lebanon,Unknown
71,Lebanon,Unknown
79,Lebanon,Unknown
80,Lebanon,Unknown


In [6]:
orgs = [node for node in G.nodes() if node in ter_df.gname.unique()]

In [7]:
W = bipartite.weighted_projected_graph(G, orgs)

In [8]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in W.nodes():
    C.add_node(n)
for e in W.edges():
    C.add_edge(e[0], e[1])
C.show('network_2.html')

In [14]:
CC = list(nx.connected_component_subgraphs(W))[0]

In [15]:
islands=island_method(CC)

In [17]:
for i in islands:
    # print the threshold level, size of the graph, and number of connected components
    print(i[0], len(i[1]), len(list(nx.connected_component_subgraphs(i[1]))))

1 53 1
2 18 1
3 9 1
4 5 1
5 5 1
6 2 1
7 2 1


In [19]:
lvl1 = islands[0][1]
lvl2 = islands[1][1]
lvl3 = islands[2][1]
lvl4 = islands[3][1]
lvl5 = islands[4][1]
lvl6 = islands[5][1]
lvl7 = islands[6][1]

In [21]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl1.nodes():
    C.add_node(n)
for e in lvl1.edges():
    C.add_edge(e[0], e[1])
C.show('island1.html')

In [22]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl2.nodes():
    C.add_node(n)
for e in lvl2.edges():
    C.add_edge(e[0], e[1])
C.show('island2.html')

In [23]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl3.nodes():
    C.add_node(n)
for e in lvl3.edges():
    C.add_edge(e[0], e[1])
C.show('island3.html')

In [24]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl4.nodes():
    C.add_node(n)
for e in lvl4.edges():
    C.add_edge(e[0], e[1])
C.show('island4.html')

In [25]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl5.nodes():
    C.add_node(n)
for e in lvl5.edges():
    C.add_edge(e[0], e[1])
C.show('island5.html')

In [26]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl6.nodes():
    C.add_node(n)
for e in lvl6.edges():
    C.add_edge(e[0], e[1])
C.show('island6.html')

In [27]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in lvl7.nodes():
    C.add_node(n)
for e in lvl7.edges():
    C.add_edge(e[0], e[1])
C.show('island7.html')